In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [3]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hobart,-42.88,147.33,48.00,81,75,5.82,AU,1594861536
1,1,castro,-24.79,-50.01,49.19,94,4,6.91,BR,1594861653
2,2,roebourne,-20.78,117.13,70.57,27,0,16.35,AU,1594861653
3,3,cabo san lucas,22.89,-109.91,91.99,55,5,11.41,MX,1594861653
4,4,isangel,-19.55,169.27,78.80,83,20,5.82,VU,1594861654


## Humidity Heatmap
Configure gmaps.

Use the Lat and Lng as locations and Humidity as the weight.

Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.

Drop any rows will null values.

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,aripuana,-9.17,-60.63,74.44,49,0,1.61,BR,1594861504
23,23,carutapera,-1.20,-46.02,75.81,90,0,6.49,BR,1594861656
35,35,maracacume,-2.04,-45.96,72.50,94,0,4.00,BR,1594861362
60,60,urucara,-2.54,-57.76,78.42,65,0,1.39,BR,1594861662
153,153,barra do garcas,-15.89,-52.26,71.04,40,0,2.64,BR,1594861632
155,155,porangatu,-13.44,-49.15,70.14,36,0,2.84,BR,1594861675
194,194,jalu,29.03,21.55,77.13,41,0,9.26,LY,1594861682
221,221,saint-louis,16.33,-15.00,78.80,74,0,2.24,SN,1594861686
230,230,lata,40.16,-8.33,73.99,74,0,3.62,PT,1594861663
258,258,sale,34.04,-6.82,75.20,88,0,8.05,MA,1594861691


## Hotel Map

Store into variable named hotel_df.

Add a "Hotel Name" column to the DataFrame.

Set parameters to search for hotels with 5000 meters.

Hit the Google Places API for each city's coordinates.

Store the first Hotel result into the DataFrame.

Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
20,aripuana,BR,-9.17,-60.63,
23,carutapera,BR,-1.20,-46.02,
35,maracacume,BR,-2.04,-45.96,
60,urucara,BR,-2.54,-57.76,
153,barra do garcas,BR,-15.89,-52.26,
155,porangatu,BR,-13.44,-49.15,
194,jalu,LY,29.03,21.55,
221,saint-louis,SN,16.33,-15.00,
230,lata,PT,40.16,-8.33,
258,sale,MA,34.04,-6.82,


In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
20,aripuana,BR,-9.17,-60.63,
23,carutapera,BR,-1.20,-46.02,Lidera
35,maracacume,BR,-2.04,-45.96,Motel imperial
60,urucara,BR,-2.54,-57.76,Pousada Maria Bonita
153,barra do garcas,BR,-15.89,-52.26,Hotel Pousada Tropical
155,porangatu,BR,-13.44,-49.15,Wembley Park Hotel
194,jalu,LY,29.03,21.55,Jalu Hotel
221,saint-louis,SN,16.33,-15.00,
230,lata,PT,40.16,-8.33,Casa Maquia
258,sale,MA,34.04,-6.82,Hôtel Mercure Rabat Sheherazade


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))